# Clustering Crypto

In [18]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [19]:
# Load the crypto_data.csv dataset.
crypto_df=pd.read_csv("crypto_data.csv", index_col=0)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [20]:
# Keep all the cryptocurrencies that are being traded.
crypto_df=crypto_df.loc[(crypto_df["IsTrading"]==True)]

In [21]:
# Remove the "IsTrading" column. 
crypto_df.drop('IsTrading', axis=1, inplace=True)
crypto_df.head()

C:\Users\mattb\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [22]:
# Remove rows that have at least 1 null value.
crypto_df.dropna()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [23]:
# Keep the rows where coins are mined.
crypto_df=crypto_df.loc[(crypto_df["TotalCoinsMined"]>0)]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [24]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names=pd.DataFrame(crypto_df['CoinName'], crypto_df.index)
crypto_names.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [25]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(['CoinName'], axis=1, inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [26]:
# Use get_dummies() to create variables for text features.
crypto_df_clean=pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'])
crypto_df_clean.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Standardize the data with StandardScaler().
crypto_scaled=StandardScaler().fit_transform(crypto_df_clean)
crypto_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [28]:
# Using PCA to reduce dimension to three principal components.
pca=PCA(n_components=3)

In [37]:
# Create a DataFrame with the three principal components.
crypto_pca=pca.fit_transform(crypto_scaled)
crypto_pca_df=pd.DataFrame(
    data=crypto_pca, columns=["principal component 1", "principal component 2", "prinicpal component 3"], index=crypto_df.index
)
crypto_pca_df

,principal component 1,principal component 2,prinicpal component 3
42,-0.330862,1.044255,-0.608059
404,-0.314185,1.044485,-0.608546
1337,2.313576,1.640884,-0.690409
BTC,-0.143502,-1.350058,0.216929
ETH,-0.155876,-2.030202,0.397654
...,...,...,...
ZEPH,2.472130,0.705664,-0.089981
GAP,-0.328908,1.044152,-0.608093
BDX,0.310223,-2.297947,0.422880
ZEN,-0.138018,-2.007135,0.395787


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [38]:
# Create an elbow curve to find the best value for K.
inertia=[]
k=list(range(1,11))

for i in k:
    km=KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca_df)
    inertia.append(km.inertia_)



C:\Users\mattb\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


In [39]:
elbow_data= {'k':k, 'inertia':inertia}
df_elbow=pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x='k', y='inertia', xticks=k)

:Curve   [k]   (inertia)

In [40]:
def get_clusters(k, data):
    # Create a copy of the DataFrame
    data = data.copy()

    # Initialize the K-Means model
    model = KMeans(n_clusters=k, random_state=0)

    # Fit the model
    model.fit(data)

    # Predict clusters
    predictions = model.predict(data)

    # Create return DataFrame with predicted clusters
    data["class"] = model.labels_

    return data

Running K-Means with `k=4`

In [41]:
# Initialize the K-Means model.
model=KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(crypto_pca_df)

# Predict clusters
predictions = model.predict(crypto_pca_df)

predictions


array([0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0,
       3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 0,
       0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 0, 3, 0, 0, 3, 0, 3, 0, 3,
       0, 3, 0, 0, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0,
       3, 0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 3, 0, 0,
       0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0,
       0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0, 3,
       0, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 0, 3, 0,

In [46]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df=pd.concat([crypto_df, crypto_pca_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName']=crypto_names['CoinName']

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class']=predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,prinicpal component 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.330862,1.044255,-0.608059,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.314185,1.044485,-0.608546,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.313576,1.640884,-0.690409,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.143502,-1.350058,0.216929,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.155876,-2.030202,0.397654,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.165021,-1.097894,-0.027445,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.400266,1.205954,-0.502825,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.139872,-2.236611,0.451656,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.154318,-2.030291,0.397629,Ethereum Classic,3
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.138017,-2.007135,0.395787,ZCash,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [70]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig=px.scatter_3d(
    clustered_df,
    x='principal component 1',
    y='principal component 2',
    z='prinicpal component 3',
    color='Class',
    hover_data=['Algorithm'],
    hover_name='CoinName',
)
fig.show()

In [74]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table()


:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,prinicpal component 3,CoinName,Class]

In [81]:
# Print the total number of tradable cryptocurrencies.
print(f'There are {len(clustered_df.index)} tradable cryptocurrencies.')

There are 532 tradable cryptocurrencies.


In [102]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
Xvar=clustered_df[['TotalCoinSupply','TotalCoinsMined']]
scaler=MinMaxScaler()

In [103]:
scaled_data=scaler.fit_transform(Xvar)

In [104]:
scaled_data

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [113]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
new_df=pd.DataFrame(scaled_data, columns = ['TotalCoinSupply','TotalCoinsMined'], index=clustered_df.index)
new_df

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
new_df['CoinName']=clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
new_df['Class']=clustered_df['Class']

new_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,0
404,5.320000e-04,0.001066,404Coin,0
1337,3.141593e-01,0.029576,EliteCoin,0
BTC,2.100000e-05,0.000018,Bitcoin,3
ETH,0.000000e+00,0.000109,Ethereum,3
LTC,8.400000e-05,0.000064,Litecoin,3
DASH,2.200000e-05,0.000009,Dash,0
XMR,0.000000e+00,0.000017,Monero,3
ETC,2.100000e-04,0.000115,Ethereum Classic,3
ZEC,2.100000e-05,0.000007,ZCash,3


In [22]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)

In [116]:
new_df.hvplot.scatter(x='TotalCoinSupply', y='TotalCoinsMined', by='Class', hover_cols='CoinName')

:NdOverlay   [Class]
   :Scatter   [TotalCoinSupply]   (TotalCoinsMined,CoinName)